In [ ]:
! pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00


In [2]:
! pip install rank_bm25

In [1]:
! pip install beir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.0/288.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [ ]:
! pip install transformers
! pip install tensorflow

In [ ]:
! pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
! pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=50cc16603ba627518d4b75974a80a6cc043a7df130e1b0d3d92d053eda16e614
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
! pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 4.1.0
    Uninstalling sentence-transformers-4.1.0:
      Successfully uninstalled sentence-transformers-4.1.0


In [3]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 47.6 MB/s eta 0:00:00


In [ ]:
!pip install transformers datasets torch

In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 8.8 MB/s eta 0:00:00


In [ ]:
#collecting all the questions from the benchmark
import json

with open('data_benchmark.json') as f:
    data_bench = json.load(f)

#print(data_bench.values())

question_bench=[]
for val in data_bench.values():
  question_bench.append(val['medical_question'])





In [ ]:
import pandas as pd
df = pd.read_csv("merged_medical_QA_dataset(2).csv")
documents = df[["question", "answer", "source"]].fillna("")

docs = [
    f"Q: {row['question']}\nA: {row['answer']}\nSource:{row['source']}"
    for _, row in df.iterrows() ]

In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss #facebook AI similarity Search, search engine for vector similarity

embedding_model = SentenceTransformer("pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")  # Biomedical embeddings ref ds favoris
# a specific embedding model trained on biomedical dataset

# Convert docs to vectors
embeddings = embedding_model.encode(docs, show_progress_bar=True)

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1]) #calcul similarité avec la distance euclidienne
index.add(embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/959 [00:00<?, ?it/s]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

query_embedding = embedding_model.encode(question_bench,show_progress_bar=True)
# Search top 10 similar docs
_, indices = index.search(query_embedding, k=10) #it was 3 before


reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

retrieved_docs = []
retrieved_sources = []
retrieved_scores= []

for i, ind_list in enumerate(indices):
    query_text = question_bench[i]

    # Créer les paires (query, doc)
    rerank_input = [(query_text, df.iloc[j]["answer"]) for j in ind_list]

    # Appliquer le reranker
    scores = reranker.predict(rerank_input)

    # Trier les documents par score décroissant
    reranked = sorted(zip(scores, ind_list), reverse=True)[:2]  # top 3

    # Reconstituer le contexte et les sources
    context = "\n".join(
        [f"Q: {df.iloc[j]['question']}\nA: {df.iloc[j]['answer']}" for _, j in reranked]
    )
    sources = list(set([df.iloc[j]["source"] for _, j in reranked]))
    top_scores = [score for score, _ in reranked]
    scaler = MinMaxScaler()
    top_scores_array = np.array(top_scores).reshape(-1, 1)
    normalized_scores = scaler.fit_transform(top_scores_array).flatten()

    retrieved_docs.append(context)
    retrieved_sources.append(sources)
    retrieved_scores.append(normalized_scores)

print(retrieved_docs[4])


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Q: What are the treatments for Peripheral Arterial Disease (P.A.D.) ?
A: Yes. In some people, lifestyle changes are not enough to control P.A.D. Surgery and other procedures may be needed. These may include bypass grafting surgery, angioplasty, a stent, or a procedure called atherectomy (ath-eh-REK-to-mee). - Your doctor may recommend bypass grafting surgery if blood flow in your limb is blocked or nearly blocked. In this type of surgery, a blood vessel from another part of the body or a man-made tube is used to make a graft. This graft bypasses (goes around) the blocked part of the artery, which allows blood to flow around the blockage. This surgery doesn't cure P.A.D., but it may increase blood flow to the affected limb.  Your doctor may recommend bypass grafting surgery if blood flow in your limb is blocked or nearly blocked. In this type of surgery, a blood vessel from another part of the body or a man-made tube is used to make a graft. This graft bypasses (goes around) the blocked

In [ ]:
import torch
from transformers import AutoTokenizer, BioGptForCausalLM
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

prompts = [f"{retrieved_docs[i]}\n\nQuestion:{question_bench[i]}\n\nAnswer:"
for i in range(len(question_bench))]
#prompt pr chaque query contenant
#1. l'info pertinente (question et réponse ds le dataset medquad)
#2. la question posé
#Answer: (vide)


tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt-large")
biogpt = BioGptForCausalLM.from_pretrained("microsoft/biogpt-large").to(device)

answers=[]
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True).to(device)
    outputs = biogpt.generate(**inputs, max_new_tokens=128)
    answer= tokenizer.decode(outputs[0], skip_special_tokens=True)#retour en format str
    answers.append(answer)




for i, (q, a) in enumerate(zip(question_bench, answers)):
     sources = ",".join(retrieved_sources[i])
     print(f"\nQuestion: {q}\nAnswer: {a}\nSources: {sources}")
     print("Relevance Scores :")
     for score in retrieved_scores[i]:
         print(f"- {score:.4f}")
     print("=" * 50)

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.29G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]


Question: Why was arsenic used to treat diseases?
Answer: Q: Do you have information about Arsenic A: Summary: Arsenic is a natural element found in soil and minerals. Arsenic compounds are used to preserve wood, as pesticides, and in some industries. Arsenic can get into air, water, and the ground from wind-blown dust. It may also get into water from runoff. You may be exposed to arsenic by - Taking in small amounts in food, drinking water, or air - Breathing sawdust or burning smoke from arsenic-treated wood - Living in an area with high levels of arsenic in rock - Working in a job where arsenic is made or used Exposure to arsenic can cause many health problems. Being exposed to low levels for a long time can change the color of your skin. It can cause corns and small warts. Exposure to high levels of arsenic can cause death. Agency for Toxic Substances Disease Registry Q: What are the treatments for Alpha-1 Antitrypsin Deficiency? A: Alpha-1 antitrypsin (AAT) deficiency has no cure

In [ ]:
import json
tout2=[]
for q,r in zip(question_bench,answers):
    data_json_biogpt_RAG = {"question": q, "answer": r}
    tout2.append(data_json_biogpt_RAG)
with open("biogpt_answer_benchmark.json", "w", encoding="utf-8") as f:
           json.dump(tout2, f, ensure_ascii=False, indent=4)

  EVALUATING THE ANSWERS AND RANKING SYSTEM WITH BLEU , ROUGE , BERTSCORE, BEIR

In [ ]:
import json
with open("biogpt_answer_benchmark.json", "r", encoding="utf-8") as f:
    data_json_biogpt_RAG = json.load(f)

answer_biogpt=[elt["answer"] for elt in data_json_biogpt_RAG]
answer_benchmark=[val['response_a'] for val in data_bench.values()]



print(len(answer_biogpt))
print(len(answer_benchmark))


400
400


In [ ]:
import evaluate

#ROUGE et BLEU évalue me chevauchement lexicale
rouge= evaluate.load("rouge")

results_rouge= rouge.compute(predictions=answer_biogpt, references=answer_benchmark, use_aggregator=True)
print(results_rouge)

{'rouge1': np.float64(0.15837066826071516), 'rouge2': np.float64(0.03974624393437065), 'rougeL': np.float64(0.09120356239292514), 'rougeLsum': np.float64(0.09160163547562666)}


In [ ]:
bleu = evaluate.load("bleu")
results_bleu = bleu.compute(predictions=answer_biogpt, references=answer_benchmark)
print(results_bleu)



{'bleu': 0.013439738100504778, 'precisions': [0.07912780224651281, 0.02001776417117858, 0.006702577841732067, 0.003073099942142254], 'brevity_penalty': 1.0, 'length_ratio': 6.9654130399080305, 'translation_length': 212062, 'reference_length': 30445}


In [ ]:
import evaluate
bertscore = evaluate.load("bertscore")
results = bertscore.compute(
    predictions=answer_biogpt,
    references=answer_benchmark,
    lang="en"
)
print(results["f1"])
print(results["precision"])
print(results["recall"])

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[0.8205773830413818, 0.7964417934417725, 0.8115816712379456, 0.819847047328949, 0.8203292489051819, 0.8354470133781433, 0.8224625587463379, 0.838950514793396, 0.8433791399002075, 0.8318220973014832, 0.8311336040496826, 0.811619758605957, 0.8325626850128174, 0.827798068523407, 0.8191922903060913, 0.8212183713912964, 0.8115466237068176, 0.8334774374961853, 0.8403564691543579, 0.8049508929252625, 0.8450709581375122, 0.8175290822982788, 0.8287767767906189, 0.828559398651123, 0.8179940581321716, 0.8249372243881226, 0.8182868957519531, 0.8113774657249451, 0.8206236362457275, 0.8431764245033264, 0.8299012184143066, 0.8241521120071411, 0.836685061454773, 0.8407658934593201, 0.8003323674201965, 0.8357447981834412, 0.8135196566581726, 0.8488567471504211, 0.819254994392395, 0.8131221532821655, 0.8351759314537048, 0.8291100263595581, 0.8450719118118286, 0.8215336203575134, 0.8132892847061157, 0.810062050819397, 0.852846622467041, 0.8359349966049194, 0.8513321280479431, 0.8213658332824707, 0.814436

In [ ]:
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from rank_bm25 import BM25Okapi
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss #facebook AI similarity Search, search engine for vector similarity
from collections import defaultdict
import numpy as np

embedding_model = SentenceTransformer("pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")  # Biomedical embeddings ref ds favoris

reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")


corpus, queries, qrels= GenericDataLoader(data_folder="retrieve_bench").load(split="test")

#prepare corpus and data
corpus_ids = list(corpus.keys())
corpus_texts = [corpus[doc_id]['text'] for doc_id in corpus_ids]
query_ids = list(queries.keys())
query_texts = [queries[query_id] for query_id in query_ids]


####### SPARSE RETRIEVAL (BM25) #######
tokenized_corpus = [doc.split() for doc in corpus_texts]
bm25 = BM25Okapi(tokenized_corpus)

sparse_results = {}
for qid, query in zip(query_ids, query_texts):
    scores = bm25.get_scores(query.split())
    ranked = np.argsort(scores)[::-1][:10]
    sparse_results[qid] = {corpus_ids[i]: float(scores[i]) for i in ranked}

####### DENSE RETRIEVAL (FAISS + BioBERT) #######
corpus_embs = embedding_model.encode(corpus_texts, batch_size=32, convert_to_numpy=True)
query_embs= embedding_model.encode(query_texts, batch_size=32, convert_to_numpy=True)

# Build FAISS index
index = faiss.IndexFlatL2(corpus_embs.shape[1])
index.add(corpus_embs)

# Perform search
k = 10
D, I = index.search(query_embs, k)
dense_results = {}
for i, qid in enumerate(query_ids):
    dense_results[qid] = {corpus_ids[j]: float(-D[i][k]) for k, j in enumerate(I[i])}  # inverser L2 pour score élevé

####### RECIPROCAL RANK FUSION (RRF) #######
def reciprocal_rank_fusion(*rankings, k=60):
    fused = defaultdict(float)
    for ranking in rankings:
        for doc_id, score in ranking.items():
            rank = list(ranking.keys()).index(doc_id) + 1
            fused[doc_id] += 1 / (k + rank)
    return dict(sorted(fused.items(), key=lambda x: x[1], reverse=True))

hybrid_results = {}
for qid in query_ids:
    sparse_ranking = sparse_results.get(qid, {})
    dense_ranking = dense_results.get(qid, {})
    fused = reciprocal_rank_fusion(sparse_ranking, dense_ranking)
    hybrid_results[qid] = dict(list(fused.items())[:10])

print("Reranking ongoing...")
rerank_results={}
for qid in tqdm(query_ids, desc="Reranking..."):
    query = queries[qid]
    doc_ids = list(hybrid_results[qid].keys())
    pairs = [(query, corpus[doc_id]["text"]) for doc_id in doc_ids]

    scores = reranker.predict(pairs)
    reranked = sorted(zip(doc_ids, scores), key=lambda x: x[1], reverse=True)

    rerank_results[qid] = {doc_id: float(score) for doc_id, score in reranked}

#evaluate with BEIR
evaluator = EvaluateRetrieval()
ndcg,_map, recall, precision= evaluator.evaluate(qrels, rerank_results, k_values=[1,3,5,10])
mrr= evaluator.evaluate_custom(qrels, rerank_results,metric="mrr",k_values=[1,3,5,10])
print("Hybrid RRF + Rerank results: \n")
for k in [1, 3, 5, 10]:
    print(f"\nMetrics @ {k}")
    print(f"NDCG@{k}:       {ndcg[f'NDCG@{k}']:.4f}")
    print(f"MRR@{k}:        {mrr[f'MRR@{k}']:.4f}")
    print(f"MAP@{k}:        {_map[f'MAP@{k}']:.4f}")
    print(f"Recall@{k}:     {recall[f'Recall@{k}']:.4f}")
    print(f"Precision@{k}:  {precision[f'P@{k}']:.4f}")





  0%|          | 0/3633 [00:00<?, ?it/s]

Reranking ongoing...


Reranking...: 100%|██████████| 323/323 [00:18<00:00, 17.62it/s]

Hybrid RRF + Rerank results: 


Metrics @ 1
NDCG@1:       0.4257
MRR@1:        0.4489
MAP@1:        0.0440
Recall@1:     0.0440
Precision@1:  0.4427

Metrics @ 3
NDCG@3:       0.3663
MRR@3:        0.5052
MAP@3:        0.0781
Recall@3:     0.0862
Precision@3:  0.3416

Metrics @ 5
NDCG@5:       0.3244
MRR@5:        0.5144
MAP@5:        0.0867
Recall@5:     0.1016
Precision@5:  0.2700

Metrics @ 10
NDCG@10:       0.2725
MRR@10:        0.5168
MAP@10:        0.0948
Recall@10:     0.1162
Precision@10:  0.1836


In [19]:
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from rank_bm25 import BM25Okapi
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss
from collections import defaultdict
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter

data_path = "retrieve_bench"
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

print(f"Corpus size: {len(corpus)} documents")
print(f"Queries: {len(queries)}")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,      # characters per chunk
    chunk_overlap=100,   # overlap between chunks
    length_function=len
)

chunked_corpus = {}
chunk_id = 0

for doc_id, doc_data in corpus.items():
    text = doc_data["text"]
    chunks = text_splitter.split_text(text)
    for chunk in chunks:
        new_id = f"{doc_id}_chunk{chunk_id}"
        chunked_corpus[new_id] = {"text": chunk, "original_doc": doc_id}
        chunk_id += 1

print(f"Chunked corpus size: {len(chunked_corpus)} passages")

# Prepare IDs and texts for retrieval
corpus_ids = list(chunked_corpus.keys())
corpus_texts = [chunked_corpus[doc_id]['text'] for doc_id in corpus_ids]

query_ids = list(queries.keys())
query_texts = [queries[qid] for qid in query_ids]

# -------------------------
# 3. Sparse retrieval (BM25)
# -------------------------
tokenized_corpus = [doc.split() for doc in corpus_texts]
bm25 = BM25Okapi(tokenized_corpus)

sparse_results = {}
for qid, query in zip(query_ids, query_texts):
    scores = bm25.get_scores(query.split())
    ranked = np.argsort(scores)[::-1][:10]
    sparse_results[qid] = {corpus_ids[i]: float(scores[i]) for i in ranked}

# -------------------------
# 4. Dense retrieval (FAISS + BioBERT)
# -------------------------
embedding_model = SentenceTransformer(
    "pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb"
)
corpus_embs = embedding_model.encode(corpus_texts, batch_size=32, convert_to_numpy=True)
query_embs = embedding_model.encode(query_texts, batch_size=32, convert_to_numpy=True)

# Build FAISS index
index = faiss.IndexFlatL2(corpus_embs.shape[1])
index.add(corpus_embs)

k = 10
D, I = index.search(query_embs, k)
dense_results = {}
for i, qid in enumerate(query_ids):
    dense_results[qid] = {corpus_ids[j]: float(-D[i][rank])
                          for rank, j in enumerate(I[i])}  # negative L2 as score

# -------------------------
# 5. Reciprocal Rank Fusion (RRF)
# -------------------------
def reciprocal_rank_fusion(*rankings, k=60):
    fused = defaultdict(float)
    for ranking in rankings:
        for doc_id, score in ranking.items():
            rank = list(ranking.keys()).index(doc_id) + 1
            fused[doc_id] += 1 / (k + rank)
    return dict(sorted(fused.items(), key=lambda x: x[1], reverse=True))

hybrid_results = {}
for qid in query_ids:
    sparse_ranking = sparse_results.get(qid, {})
    dense_ranking = dense_results.get(qid, {})
    fused = reciprocal_rank_fusion(sparse_ranking, dense_ranking)
    hybrid_results[qid] = dict(list(fused.items())[:10])

# -------------------------
# 6. Cross-Encoder Reranking
# -------------------------
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
rerank_results = {}

print("Reranking ongoing...")
for qid in tqdm(query_ids, desc="Reranking"):
    query = queries[qid]
    doc_ids = list(hybrid_results[qid].keys())
    pairs = [(query, chunked_corpus[doc_id]["text"]) for doc_id in doc_ids]

    scores = reranker.predict(pairs)
    reranked = sorted(zip(doc_ids, scores), key=lambda x: x[1], reverse=True)

    # Map chunk IDs back to original document IDs
    rerank_results[qid] = {}
    for chunk_id, score in reranked:
        orig_doc_id = chunked_corpus[chunk_id]["original_doc"]
        rerank_results[qid][orig_doc_id] = max(
            rerank_results[qid].get(orig_doc_id, 0.0),
            float(score)
        )

# -------------------------
# 7. Evaluate with BEIR
# -------------------------
evaluator = EvaluateRetrieval()
ndcg, _map, recall, precision = evaluator.evaluate(qrels, rerank_results, k_values=[1, 3, 5, 10])
mrr = evaluator.evaluate_custom(qrels, rerank_results, metric="mrr", k_values=[1, 3, 5, 10])

print("Hybrid RRF + Rerank + Chunking results: \n")
for k in [1, 3, 5, 10]:
    print(f"\nMetrics @ {k}")
    print(f"NDCG@{k}:       {ndcg[f'NDCG@{k}']:.4f}")
    print(f"MRR@{k}:        {mrr[f'MRR@{k}']:.4f}")
    print(f"MAP@{k}:        {_map[f'MAP@{k}']:.4f}")
    print(f"Recall@{k}:     {recall[f'Recall@{k}']:.4f}")
    print(f"Precision@{k}:  {precision[f'P@{k}']:.4f}")

  0%|          | 0/3633 [00:00<?, ?it/s]

Corpus size: 3633 documents
Queries: 323
Chunked corpus size: 10259 passages
Reranking ongoing...


Reranking: 100%|██████████| 323/323 [00:06<00:00, 48.78it/s]

Hybrid RRF + Rerank + Chunking results: 


Metrics @ 1
NDCG@1:       0.3762
MRR@1:        0.4613
MAP@1:        0.0431
Recall@1:     0.0431
Precision@1:  0.3870

Metrics @ 3
NDCG@3:       0.3269
MRR@3:        0.5134
MAP@3:        0.0740
Recall@3:     0.0839
Precision@3:  0.2993

Metrics @ 5
NDCG@5:       0.3056
MRR@5:        0.5244
MAP@5:        0.0841
Recall@5:     0.1013
Precision@5:  0.2607

Metrics @ 10
NDCG@10:       0.2540
MRR@10:        0.5256
MAP@10:        0.0886
Recall@10:     0.1126
Precision@10:  0.1684
